## LangChain LCEL Chain + Pydantic Output Parser

### Install Libs

In [1]:
!pip install -Uq langchain langchain_openai

### Import Libs

In [2]:
from typing import List

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI

### Initiate OpenAI LLM

In [3]:
model = ChatOpenAI(temperature=0.9)

### Create Pydantic objects and chain

In [4]:
# Define your desired data structure.
class FilmRecommendation(BaseModel):
    genre: str = Field(description="Recommended film genre")
    film_name: str = Field(description="Recommended film name")


# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=FilmRecommendation)

prompt = PromptTemplate(
    template="""
    You are the best film recommender based on the user query. Users tell you a scene and you recommend a movie and its genre based on that.
    
    {format_instructions}
    
    Scene: {scene}""",
    input_variables=["scene"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Chain ->
chain = prompt | model | parser



### Run

In [5]:
list_scenes = [
    'Floating ship middle of the ocean',
    'A superhero crawling the tower',
    'travel in the time',
]


for scene in list_scenes:
    print(f'{scene}: {chain.invoke({"scene": scene})} \n{"-"*40}')

Floating ship middle of the ocean: genre='Adventure' film_name='Pirates of the Caribbean: The Curse of the Black Pearl' 
----------------------------------------
A superhero crawling the tower: genre='Action' film_name='Spider-Man: Homecoming' 
----------------------------------------
travel in the time: genre='Sci-Fi' film_name='Back to the Future' 
----------------------------------------


In [6]:
result = chain.invoke({"scene": list_scenes[-1]})

print(f'Genre: {result.genre}')
print(f'Film Name: {result.film_name}')

result

Genre: Science Fiction
Film Name: Back to the Future


FilmRecommendation(genre='Science Fiction', film_name='Back to the Future')